In [1]:
import torch
import numpy as np
import os
from torch import nn
import sys
import polars as pl
import sklearn
import shap
from functools import partial
import pandas as pd
import random
from matplotlib import pyplot as plt
import time
from rdkit.Chem import DataStructs
from tqdm import tqdm
from joblib import Parallel, delayed
import math
sys.path.append('../')


from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordLevel, BPE
from tokenizers.pre_tokenizers import Whitespace,Split,ByteLevel, WhitespaceSplit
from tokenizers.normalizers import Lowercase, NFKC
import os
import polars as pl
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
from tqdm import tqdm
import time
import json
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordLevelTrainer, WordPieceTrainer
import gc
from transformers import AutoConfig, AutoTokenizer, AutoModel, DataCollatorWithPadding
import mapply
from collections import Counter
from rdkit import Chem
from rdkit.Chem import AllChem
from functools import partial

multiprocessing.cpu_count()

80

In [6]:
test_feat = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ecfp6/test.npy')
test_feat = np.unpackbits(test_feat, axis = -1)
print(test_feat.shape)
constant_col_idxs = np.where(test_feat.min(axis = 0) == test_feat.max(axis = 0))[0]
print(len(constant_col_idxs))
constant_col_idxs

(878022, 2048)
8


array([ 650,  807, 1152, 1182, 1380, 1621, 1917, 2007])

In [7]:
test_feat = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/topological_torsion/test.npy')
test_feat = np.unpackbits(test_feat, axis = -1)
print(test_feat.shape)
constant_col_idxs = np.where(test_feat.min(axis = 0) == test_feat.max(axis = 0))[0]
print(len(constant_col_idxs))
constant_col_idxs

(878022, 2048)
668


array([   2,    3,    7,   10,   11,   15,   18,   19,   21,   22,   23,
         26,   27,   31,   35,   38,   39,   40,   41,   42,   43,   45,
         46,   47,   53,   54,   55,   56,   57,   58,   59,   62,   63,
         65,   66,   67,   71,   75,   76,   77,   78,   79,   81,   82,
         83,   86,   87,   90,   91,   94,   95,   99,  103,  105,  106,
        107,  111,  115,  127,  131,  135,  136,  137,  138,  139,  143,
        145,  146,  147,  150,  151,  155,  156,  157,  158,  159,  162,
        163,  167,  170,  171,  174,  175,  183,  186,  187,  191,  194,
        195,  198,  199,  203,  206,  207,  208,  209,  210,  211,  214,
        215,  223,  226,  227,  230,  231,  247,  251,  255,  257,  258,
        259,  267,  270,  271,  275,  279,  287,  290,  291,  294,  295,
        298,  299,  300,  301,  302,  303,  307,  311,  319,  325,  326,
        327,  331,  335,  339,  343,  347,  351,  357,  358,  359,  362,
        363,  366,  367,  371,  387,  391,  394,  3

In [8]:
test_feat = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/mhfp/test.npy')
test_feat = np.unpackbits(test_feat, axis = -1)
print(test_feat.shape)
constant_col_idxs = np.where(test_feat.min(axis = 0) == test_feat.max(axis = 0))[0]
print(len(constant_col_idxs))
constant_col_idxs

(878022, 2048)
1


array([645])

In [14]:
ais_train_tok = AutoTokenizer.from_pretrained('/home/dangnh36/datasets/competitions/leash_belka/processed/tokenizer/ais_train/')
print(len(ais_train_tok))
ais_train_tokens = list(ais_train_tok.get_vocab().keys())
ais_train_tokens

210


['[C;!R;COO]',
 '[C;R;CCCO]',
 '[s;R;CN]',
 '[CH2;R;NS]',
 '[n;R;CO]',
 '[O;R;CC]',
 '[C;R;CCCC]',
 '[C;R;CCCS]',
 '[N;!R;CN]',
 '[o;R;CC]',
 '[[N-];!R;N]',
 '[C;!R;CN]',
 '[c;R;CClS]',
 '[O;!R;C]',
 '[C;!R;CCCO]',
 '[CH2;!R;CS]',
 '[C;R;CCOO]',
 '[CH;R;CCO]',
 '[[C@@H];!R;CCN]',
 '[[C@H];R;CCC]',
 '[N;R;CCS]',
 '[C;!R;CCO]',
 '[[C@@H];R;CCN]',
 '[c;R;BrCS]',
 '5',
 '=',
 '[C;R;CNO]',
 '[NH;R;CC]',
 '[c;R;CFN]',
 '6',
 '[NH;!R;CDy]',
 '[c;R;CCN]',
 '[O;!R;S]',
 '[SH;!R;C]',
 '[Dy;!R;N]',
 '[C;!R;CClCl]',
 '[[C@H];!R;CCN]',
 '[Cl;!R;C]',
 '[c;R;BCC]',
 '[B;R;COO]',
 '[S;R;CCO]',
 '[C;!R;CFFF]',
 '[C;R;CCCF]',
 '[CH2;R;CS]',
 '[CH3;!R;C]',
 '[N;!R;CCS]',
 '[c;R;CNO]',
 '[nH;R;NN]',
 '[NH;!R;CC]',
 '[CH;R;CCS]',
 '[C;!R;CNO]',
 '[CH;!R;CC]',
 '[c;R;ClNS]',
 '[Si;!R;CCCC]',
 '[c;R;CNS]',
 '[CH;!R;CCC]',
 '[O;R;CN]',
 '[N;R;CCC]',
 '[c;R;COO]',
 '[o;R;NN]',
 '[CH;!R;FFO]',
 '[[C@@H];!R;CCO]',
 '[NH2;!R;C]',
 '[n;R;CN]',
 '[[C@H];R;CCO]',
 '[N;R;CCO]',
 '[CH2;!R;CN]',
 '[CH2;!R;C]',
 '[S;R;C

In [15]:
ais_test_tok = AutoTokenizer.from_pretrained('/home/dangnh36/datasets/competitions/leash_belka/processed/tokenizer/ais_test/')
print(len(ais_test_tok))
ais_test_tokens = list(ais_test_tok.get_vocab().keys())
ais_test_tokens

224


['[cH;R;CS]',
 '[CH;R;CCO]',
 '[CH2;R;CN]',
 '[F;!R;C]',
 '[C;!R;CCO]',
 '[c;R;CCC]',
 '[[C@H];R;CCO]',
 '[O;R;CC]',
 '[C;!R;CCCl]',
 '[O;!R;CC]',
 '[[N+];!R;NN]',
 '[CH;R;CCN]',
 '[C;R;COO]',
 '1',
 '[[C@@H];R;CCC]',
 '[C;!R;COO]',
 '[[C@@H];R;CCO]',
 '[CH2;!R;NN]',
 '[[C@];R;CCCN]',
 '[c;R;COO]',
 '[n;R;CCC]',
 '4',
 '[nH;R;NN]',
 '[nH;R;CN]',
 '[CH3;!R;Si]',
 '[CH;!R;CFF]',
 '[[C@H];R;CCC]',
 '9',
 '[CH2;!R;CC]',
 '=',
 '[CH3;!R;C]',
 '[S;!R;CC]',
 '[C;!R;CSi]',
 '[C;!R;CCC]',
 '[CH;R;CN]',
 '[C;R;CCO]',
 '[[C@@];R;CCCN]',
 '6',
 '[C;R;CNN]',
 '[S;!R;NNOO]',
 '[C;R;CCC]',
 '[CH2;!R;CF]',
 '[N;!R;CCC]',
 '[CH2;R;CS]',
 '[[C@H];!R;CCO]',
 '[CH2;R;OO]',
 '[NH;!R;CC]',
 '[CH;R;CO]',
 '[N;R;CCC]',
 '[C;R;CCCC]',
 '[S;!R;CCOO]',
 '[C;!R;CNS]',
 '[cH;R;CO]',
 '[C;!R;CCCC]',
 '[[C@@H];!R;CCO]',
 '[S;!R;CNOO]',
 '[C;R;CCFF]',
 '[N;R;CCN]',
 '[[N-];!R;N]',
 '[C;!R;CNO]',
 '[CH;!R;FFN]',
 '[C;R;CCS]',
 '-',
 '[cH;R;NO]',
 '[[C@@];R;CCCC]',
 '[C;!R;CNN]',
 '[S;R;CNOO]',
 '[CH;R;CC]',
 '[s;R;CN]

In [17]:
saved_tokenizer = PreTrainedTokenizerFast(
        model_max_length = 512,
        padding_side = 'right',
        tokenizer_file = '/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/tokenizer.json',
        unk_token='[UNK]',
        pad_token = '[PAD]',
)
print(len(saved_tokenizer))
saved_test_tokens = list(saved_tokenizer.get_vocab().keys())
saved_tokenizer

224


PreTrainedTokenizerFast(name_or_path='', vocab_size=224, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
set(ais_test_tokens).difference(set(ais_train_tokens))

{'[C;!R;CNS]',
 '[C;R;CNN]',
 '[C;R;FFOO]',
 '[CH2;!R;NN]',
 '[CH2;!R;NO]',
 '[CH;!R;CCF]',
 '[CH;R;CCF]',
 '[CH;R;CN]',
 '[CH;R;CO]',
 '[S;!R;NNOO]',
 '[[C@@];R;CCCC]',
 '[[C@];R;CCCC]',
 '[c;R;CSS]',
 '[c;R;NOO]'}

In [18]:
set(saved_test_tokens).difference(set(ais_train_tokens))

{'[C;!R;CNS]',
 '[C;R;CNN]',
 '[C;R;FFOO]',
 '[CH2;!R;NN]',
 '[CH2;!R;NO]',
 '[CH;!R;CCF]',
 '[CH;R;CCF]',
 '[CH;R;CN]',
 '[CH;R;CO]',
 '[S;!R;NNOO]',
 '[[C@@];R;CCCC]',
 '[[C@];R;CCCC]',
 '[c;R;CSS]',
 '[c;R;NOO]'}

In [19]:
set(saved_test_tokens).difference(set(ais_train_tokens)) == set(ais_test_tokens).difference(set(ais_train_tokens))

True

In [7]:
set(ais_train_tokens).difference(set(ais_test_tokens))

set()

In [22]:
ais_test_vocab = ais_test_tok.get_vocab()
unk_token_ids = [
    saved_tokenizer.get_vocab()[k] for k in set(saved_test_tokens).difference(set(ais_train_tokens))
]
unk_token_ids

[194, 220, 155, 111, 197, 161, 105, 198, 146, 162, 195, 219, 151, 163]

In [10]:
ais_test_feat = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/test.npy')
ais_test_feat.shape

(878022, 115)

In [34]:
ais_train_feat = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/train.npy', mmap_mode='r')[:10_000_000]
ais_train_feat.shape

(10000000, 126)

In [35]:
for i, unk_tok_id in enumerate(unk_token_ids):
    print(i, unk_tok_id)
    assert np.any(ais_test_feat==unk_tok_id)
    assert np.all(ais_train_feat != unk_tok_id)

0 194
1 220
2 155
3 111
4 197
5 161
6 105
7 198
8 146
9 162
10 195
11 219
12 151
13 163


In [33]:
sorted([194, 220, 155, 111, 197, 161, 105, 198, 146, 162, 195, 219, 151, 163])

[105, 111, 146, 151, 155, 161, 162, 163, 194, 195, 197, 198, 219, 220]

In [39]:
saved_tokenizer.get_vocab()['[PAD]']

0